<a href="https://colab.research.google.com/github/vuong-viet-hung/Electrical-Component-Recognition/blob/main/SqueezeNet/Electronic_Component_Recognition_Using_SqueezeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Electronic Component Recognition Using SqueezeNet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd "/content/drive/MyDrive/Mandevices/Machine Learning/Electronic Component Recognition/SqueezeNet"

/content/drive/MyDrive/Mandevices/Machine Learning/Electronic Component Recognition/SqueezeNet


## A. Prepare the dataset

In [ ]:
import shutil
from pathlib import Path
from collections import defaultdict
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Dropout, GlobalAveragePooling2D, MaxPooling2D, Input 

In [ ]:
DATASET_PATH = Path('../Dataset/')
TRAIN_PATH = Path('../Dataset/Train/')
VALID_PATH = Path('../Dataset/Valid/')
TEST_PATH = Path('../Dataset/Test/'
NUM_CLASSES = 12
IMAGE_SIZE = (224, 224)
NUM_CHANNELS = 3
BATCH_SIZE = 32

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1 / 255)
valid_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1 / 255)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1 / 255)

In [ ]:
train_loader = train_generator.flow_from_directory(
    directory=str(TRAIN_PATH),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
) 

valid_loader = valid_generator.flow_from_directory(
    directory=str(VALID_PATH),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
) 

test_loader = test_generator.flow_from_directory(
    directory=str(TEST_PATH),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
) 

Found 8884 images belonging to 12 classes.
Found 1117 images belonging to 12 classes.
Found 1105 images belonging to 12 classes.


## B. Build the model

In [ ]:
def fire(x, s1x1_squeeze, e1x1_expand, e3x3_expand):
    s1x1 = tf.keras.layers.Conv2D(filters=s1x1_squeeze, kernel_size=1, padding='same', activation='relu')(x)
    e1x1 = tf.keras.layers.Conv2D(filters=e1x1_expand, kernel_size=1, padding='same', activation='relu')(s1x1)
    e3x3 = tf.keras.layers.Conv2D(filters=e3x3_expand, kernel_size=3, padding='same', activation='relu')(s1x1)
    return tf.keras.layers.concatenate([e1x1, e3x3])

In [ ]:
def Fire(s1x1_squeeze, e1x1_expand, e3x3_expand):
    return lambda x: fire(x, s1x1_squeeze, e1x1_expand, e3x3_expand)

In [ ]:
input = Input(shape=[*IMAGE_SIZE, NUM_CHANNELS])
conv1 = Conv2D(filters=96, kernel_size=7, strides=2, padding='same', activation='relu')(input)
maxpool1 = MaxPooling2D(pool_size=3, strides=2)(conv1)
fire2 = Fire(16, 64, 64)(maxpool1)
fire3 = Fire(16, 64, 64)(fire2)
fire4 = Fire(32, 128, 128)(fire3)
maxpool4 = MaxPooling2D(pool_size=3, strides=2)(fire4)
fire5 = Fire(32, 128, 128)(maxpool4)
fire6 = Fire(48, 192, 192)(fire5)
fire7 = Fire(48, 192, 192)(fire6)
fire8 = Fire(64, 256, 256)(fire7)
maxpool8 = MaxPooling2D(pool_size=3, strides=2)(fire8)
fire9 = Fire(64, 256, 256)(maxpool8)
dropout9 = Dropout(0.5)
conv10 = Conv2D(filters=NUM_CLASSES, kernel_size=1, strides=1, padding='same', activation='relu')(fire9)
avgpool10 = GlobalAveragePooling2D()(conv10)
softmax = Dense(NUM_CLASSES, activation='softmax')(avgpool10)

In [ ]:
model = tf.keras.Model(input, softmax)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 96  14208       ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 55, 55, 96)   0           ['conv2d[0][0]']                 
                                                                                              

## C. Train the model

In [ ]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [ ]:
history = model.fit(train_loader, validation_data=valid_loader, epochs=100, callbacks=[callback])

Epoch 1/100
278/278 [==============================] - 57s 204ms/step - loss: 1.1966 - accuracy: 0.5531 - val_loss: 1.0606 - val_accuracy: 0.5918
Epoch 2/100
278/278 [==============================] - 57s 204ms/step - loss: 1.0103 - accuracy: 0.6012 - val_loss: 0.8466 - val_accuracy: 0.6714
Epoch 3/100
278/278 [==============================] - 57s 206ms/step - loss: 0.8138 - accuracy: 0.6824 - val_loss: 0.6609 - val_accuracy: 0.7422
Epoch 4/100
278/278 [==============================] - 57s 204ms/step - loss: 0.6786 - accuracy: 0.7484 - val_loss: 0.8192 - val_accuracy: 0.7153
Epoch 5/100
278/278 [==============================] - 57s 206ms/step - loss: 0.5545 - accuracy: 0.8009 - val_loss: 0.4405 - val_accuracy: 0.8281
Epoch 6/100
278/278 [==============================] - 57s 206ms/step - loss: 0.4900 - accuracy: 0.8215 - val_loss: 0.4382 - val_accuracy: 0.8451
Epoch 7/100
278/278 [==============================] - 57s 205ms/step - loss: 0.4405 - accuracy: 0.8475 - val_loss: 0.3773 -

## D. Evaluate the model

In [ ]:
model.evaluate(test_loader)

35/35 [==============================] - 167s 5s/step - loss: 0.1028 - accuracy: 0.9774


[0.10276728868484497, 0.9773755669593811]

## E. Save the model

In [ ]:
%mkdir 'saved_model'

In [ ]:
model.save('save_model/model.h5')